In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv
from sklearn.model_selection import train_test_split
from sysidentpy.model_structure_selection import FROLS
from sysidentpy.basis_function._basis_function import Polynomial
from sysidentpy.metrics import root_relative_squared_error
from sysidentpy.utils.generate_data import get_siso_data
from sysidentpy.utils.display_results import results
from sysidentpy.utils.plotting import plot_residues_correlation, plot_results
from sysidentpy.residues.residues_correlation import compute_residues_autocorrelation, compute_cross_correlation

### Data 
Using data from 28R30S model simulation, 'U_core_T4B_08June23.csv'.<br>
Input (30) : Current of stator 1 to 30 <br>
Output(30) : Torque of stator 1 to 30. 

In [4]:

url = "C:/Users/mfavre4/Desktop/System_Identification/Python_Package/Data/U_core_T4B_08June23.csv"
df = pd.read_csv(url, sep=",")
dataset = df.astype(float)

#Inputs - Current of each poles
ds_I = (dataset.loc[:,"I_1":"I_30"])

#Output - Total torque
frames = [(dataset.loc[0:701,"totalTorque_0":"totalTorque_29"]),(dataset.loc[0:701,'totalTorque_30']),(dataset.loc[0:701,'Total_Torque']),(dataset.loc[0:701,'Solved']) ]
ds_T = pd.concat(frames,axis=1)

print(ds_T.shape)

I_id, I_val, xlags = [None for n in range(ds_I.shape[1])],[None for n in range(ds_I.shape[1])],[None for n in range(ds_I.shape[1])]
T_id, T_val, ylags = [None for n in range(ds_T.shape[1])],[None for n in range(ds_T.shape[1])],[None for n in range(ds_T.shape[1])]



#---------------Input setup
for i,c in enumerate(ds_I):
    #Creating id arrays with the current of each pole
    exec(str(c)+'_id'+'= ds_I[str(c)][0:492].values.reshape(-1, 1)')
    exec('I_id[i]='+str(c)+'_id')
    
    #Creating validation arrays with the current of each pole
    exec(str(c)+'_val'+'= ds_I[str(c)][492:702:].values.reshape(-1, 1)')
    exec('I_val[i]='+str(c)+'_val')

#Input id and validation sets
x_id = np.concatenate([x for x in I_id], axis =1 )  
x_val = np.concatenate([x for x in I_val], axis =1 ) 

#Output id and validation sets
y_id, y_val = ds_T['Total_Torque'][0:492].values.reshape(-1, 1), ds_T['Total_Torque'][492::].values.reshape(-1, 1)

#Lags for each inputs [[1,2], [1,2]...]
for i,x in enumerate(xlags): xlags[i] = ([i for i in range(1,2)])
    
    
    
#---------------Output setup
for i,c in enumerate(ds_T):
    #Creating id arrays with the current of each pole
    exec(str(c)+'_id'+'= ds_T[str(c)][0:492].values.reshape(-1, 1)')
    exec('T_id[i]='+str(c)+'_id')
    
    #Creating validation arrays with the current of each pole
    exec(str(c)+'_val'+'= ds_T[str(c)][492:702:].values.reshape(-1, 1)')
    exec('T_val[i]='+str(c)+'_val')
    
y_id = np.concatenate([x for x in T_id], axis =1 )  
y_val = np.concatenate([x for x in T_val], axis =1 ) 


#ylags = [i for i in range(0,ds_T.shape[1])]
#np.asarray(ylags, dtype=np.float32)
for i,x in enumerate(ylags): ylags[i] = ([i for i in range(1,3)])
print(ylags)



(702, 33)
[[1, 2], [1, 2], [1, 2], [1, 2], [1, 2], [1, 2], [1, 2], [1, 2], [1, 2], [1, 2], [1, 2], [1, 2], [1, 2], [1, 2], [1, 2], [1, 2], [1, 2], [1, 2], [1, 2], [1, 2], [1, 2], [1, 2], [1, 2], [1, 2], [1, 2], [1, 2], [1, 2], [1, 2], [1, 2], [1, 2], [1, 2], [1, 2], [1, 2]]


In [5]:
deg = 1
basis_function = Polynomial(degree=deg)
model = FROLS(
    order_selection=True,
    n_info_values=10,
    extended_least_squares=False,
    ylag=ylags,
    xlag=xlags,
    info_criteria='aic',
    estimator='least_squares',
    basis_function=basis_function
)
model.fit(X=x_id, y=y_id)



C:\Users\mfavre4\AppData\Local\anaconda3\Lib\site-packages\sysidentpy\utils\deprecation.py:37: FutureWarning: Passing a string to define the estimator will rise an error in v0.4.0. 
 You'll have to use FROLS(estimator=LeastSquares()) instead. 
 The only change is that you'll have to define the estimator first instead of passing a string like 'least_squares'. 
 This change will make easier to implement new estimators and it'll improve code readability.
  warnings.warn(message, FutureWarning)


TypeError: unsupported operand type(s) for -: 'int' and 'list'

In [6]:

yhat = model.predict(X=x_val, y=y_val)
rrse = root_relative_squared_error(y_val, yhat)

print('Polynomial degree: '+str(deg))
print('Input lags: '+str(xlags[0]))
print('Output lags: '+str(ylags))
print('RRSE: '+str(round(rrse,3)))
print(' ')

r = pd.DataFrame(
    results(
        model.final_model, model.theta, model.err,
        model.n_terms, err_precision=8, dtype='sci'
        ),
    columns=['Regressors', 'Parameters', 'ERR'])
print(r)
plot_results(y=y_val[0:100], yhat=yhat[0:100], n=1000)


TypeError: 'NoneType' object is not iterable